<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Gathering Intelligence with Generative AI   

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.29.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.29.0-alpha"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info using o1 series of models
var azureOpenAIEndpoint = config["AzureOpenAI:o1Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:o1APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:o1ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.0 Microsoft.Extensions.Configuration.Json, 9.0.0 Microsoft.SemanticKernel, 1.29.0 Microsoft.SemanticKernel.Plugins.Core, 1.29.0-alpha

In [2]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
// #r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.0"
// #r "nuget: Microsoft.SemanticKernel, 1.29.0"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Using Azure OpenAI Service


### Step 2 - Understanding Built-In Decision Intelligence with Generative AI

Generative AI models are trained on very large amounts of data. This data includes decision-making techniques, decision frameworks, reasoning techniques, decision best practices and other adjacent decision data that can help make decisions. Therefore, an obvious step in gathering Decision Intelligence that built-into the GenAI model is important to understand.  

Certain models (typically those that have more parameters) have more "intelligence built-in". Furthermore, models that specialize in certain tasks or a fine-tuned for a specific domain can also have more intelligence built-in. A great example of model specialization is the OpenAI o1 series of models. The o1 series of models are advanced reasoning models that are a great fit for decision-making. Therefore, it is a good hypotheses to test if these o1 models have additional Decision Intelligence data built-in.  

Execute the cell below to view an exhaustive list of Decision Frameworks built-into the o1 series of models.

In [3]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";
var decisionAnalysisPrompt = """
Identify and a very complete list of decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each framework supports better analysis and reasoning in different decision-making scenarios.

Create a column for the Number (in ascending order), framework name and decsription.
Surface the gathered frameworks in a Markdown table format. Do not surround the markdown with triple backticks.
Only return the Markdown table no other text.
""";

// How the prompt looks like to the LLM
var simpleDecisionPromptTemplate = $"""
System Prompt: {systemPromptForDecisions}

Request from the user: {decisionAnalysisPrompt}
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(decisionAnalysisPrompt);
Console.WriteLine(simplePromptResponse.GetValue<string>());

| Number | Framework Name                          | Description                                                                                                                                                                                                                                                                                                                                                 |
|--------|-----------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 1      | SWOT Analysis                           | Evaluates Strengths, Weaknesses, Opportunities, and Threats to identify internal and external factors affecting decisions. Helps in strategic

| Number | Framework Name            | Description                                                                                                                                                                  |
|--------|---------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 1      | SWOT Analysis             | Identifies Strengths, Weaknesses, Opportunities, and Threats to assess internal and external factors, enhancing strategic planning and decision-making.                       |
| 2      | Decision Matrix           | Evaluates and prioritizes options by scoring them based on set criteria, bringing objectivity to decisions involving multiple factors.                                        |
| 3      | Cost-Benefit Analysis     | Compares expected costs and benefits of actions to determine financial feasibility and profitability, aiding in economic decision-making.                                    |
| 4      | Pareto Analysis           | Applies the 80/20 rule to identify the most significant factors contributing to an outcome, helping prioritize issues with the greatest impact when addressed.               |
| 5      | Six Thinking Hats         | Assigns different perspectives (hats) to approach decisions from various angles, encouraging comprehensive analysis and creative thinking in group discussions.              |
| 6      | OODA Loop                 | Stands for Observe, Orient, Decide, Act; a cyclical process emphasizing agility and situational awareness, useful in fast-paced and competitive scenarios.                   |
| 7      | PDCA Cycle                | Plan-Do-Check-Act; a continuous improvement framework that implements changes systematically, ensuring decisions are tested and refined through iterations.                   |
| 8      | Root Cause Analysis       | Identifies the fundamental causes of problems rather than just symptoms, leading to more effective and lasting solutions.                                                     |
| 9      | Fishbone Diagram          | Visually maps out potential causes of a problem, facilitating thorough analysis and problem-solving by categorizing contributing factors.                                     |
| 10     | PEST Analysis             | Assesses Political, Economic, Social, and Technological external factors impacting decisions or organizations, aiding in strategic planning and environmental scanning.        |
| 11     | Kepner-Tregoe Method      | Provides a structured approach for gathering information, prioritizing factors, and systematically choosing the best option in complex decisions.                             |
| 12     | Force Field Analysis      | Identifies driving and restraining forces affecting a decision, helping visualize factors that can impact success and guide strategies to strengthen or weaken these forces.  |
| 13     | Risk Analysis             | Involves identifying and assessing potential risks and uncertainties in decisions, allowing for better preparation and mitigation strategies.                                 |
| 14     | Decision Trees            | Uses a graphical representation of choices and their possible consequences, including chance event outcomes, resource costs, and utility, to aid in selecting the most beneficial option. |
| 15     | Analytic Hierarchy Process (AHP) | Breaks down complex decisions into a hierarchy of more manageable parts, using pairwise comparisons and priority scaling to determine the best option based on multiple criteria. |
| 16     | Scenario Analysis         | Explores and analyzes possible future events by considering alternative plausible scenarios, assisting in strategic planning and risk management under uncertainty.          |
| 17     | Delphi Technique          | Utilizes rounds of questionnaires sent to experts to gather opinions and reach a consensus on complex issues, enhancing decision-making with expert insights.                  |
| 18     | SMART Goals               | Ensures goals are Specific, Measurable, Achievable, Relevant, and Time-bound, facilitating clearer objectives and effective planning.                                         |
| 19     | Monte Carlo Simulation    | Uses random sampling and statistical modeling to estimate mathematical functions and mimic the operations of complex systems, assessing risk and uncertainty in decision-making. |
| 20     | Bayes' Theorem            | Applies probabilistic reasoning to update the probability for a hypothesis as more evidence or information becomes available, enhancing decisions under uncertainty.           |
| 21     | Game Theory               | Analyzes strategic interactions where the outcome for each participant depends on the actions of others, aiding in competitive and cooperative decision-making scenarios.      |
| 22     | Value Chain Analysis      | Examines an organization's activities to understand how they contribute to value creation, helping identify areas for improvement and competitive advantage.                   |
| 23     | Balanced Scorecard        | Integrates financial and non-financial performance measures to provide a more comprehensive view of organizational performance against strategic goals.                       |
| 24     | The Ladder of Inference   | Describes the mental process of moving from data to decisions, highlighting the importance of testing assumptions and reasoning to avoid flawed conclusions.                  |
| 25     | Pros-Cons Analysis        | Lists the advantages and disadvantages of each option, facilitating straightforward comparison and aiding in rational decision-making.                                        |
| 26     | Effective Altruism Framework | Utilizes evidence and reasoning to determine the most effective ways to benefit others, guiding philanthropic and ethical decisions toward the greatest impact.            |
| 27     | Eisenhower Matrix         | Prioritizes tasks based on urgency and importance, helping focus on what matters most and effectively manage time by categorizing tasks into four quadrants.                   |
| 28     | The Cynefin Framework     | Helps understand the context of a situation (simple, complicated, complex, chaotic) to make appropriate decisions and apply suitable management approaches.                    |
| 29     | Systems Thinking          | Views problems as part of an overall system, focusing on the interactions and interdependencies rather than isolated components, leading to more comprehensive solutions.       |
| 30     | Cognitive Bias Awareness  | Recognizes and mitigates cognitive biases that can impair judgment, leading to more rational, objective decisions by acknowledging personal and systemic biases.               |
| 31     | RACI Matrix               | Clarifies roles and responsibilities by defining who is Responsible, Accountable, Consulted, and Informed in decision-making processes, enhancing communication and accountability. |
| 32     | Causal Loop Diagrams      | Uses visual representations of systems to show feedback loops and interactions, aiding in understanding complex systems and predicting behavior over time.                     |
| 33     | TRIZ (Theory of Inventive Problem Solving) | Provides a systematic approach for solving complex problems by analyzing patterns in innovative solutions across industries.                             |
| 34     | MOSCOW Method             | Prioritizes requirements or tasks by categorizing them as Must-have, Should-have, Could-have, and Won't-have, assisting in resource allocation and scope management.           |
| 35     | SCQA Framework            | Stands for Situation, Complication, Question, and Answer; helps structure communication and decision narratives effectively, ensuring clarity and logical flow.                |
| 36     | Fail-Safing (Poka-Yoke)   | Designs processes to prevent errors or mitigate their effects, enhancing decision robustness and operational reliability by anticipating potential failures.                   |
| 37     | Benchmarking              | Compares practices and performance metrics with industry leaders or competitors to identify areas for improvement and set performance goals.                                   |
| 38     | Stakeholder Analysis      | Identifies stakeholders' interests, influence, and impact on decisions, aiding in effective communication, conflict resolution, and ensuring decisions meet key stakeholders' needs. |
| 39     | Critical Path Method (CPM)| Determines the sequence of essential tasks that determine the minimum project duration, optimizing scheduling, resource allocation, and timely decision-making in projects.     |
| 40     | Ethical Decision-Making Models | Incorporate ethical principles and values into the decision process, ensuring choices are aligned with moral and societal standards, and considering the broader impact.       |
| 41     | Nudge Theory              | Applies behavioral economics insights to subtly influence choices and behaviors, improving decision outcomes without restricting options by altering the decision environment.    |
| 42     | Red Teaming               | Involves adopting an adversarial approach to challenge strategies and decisions, exposing vulnerabilities, and enhancing robustness by anticipating opposition or failure modes. |
| 43     | Design Thinking           | Employs a human-centered approach to innovation, integrating the needs of people, possibilities of technology, and requirements for business success through iterative prototyping and testing. |
| 44     | Empathy Mapping           | Uses visual tools to understand users' experiences and perspectives, promoting decisions that are aligned with stakeholders' needs and enhancing user-centered design.            |
| 45     | McKinsey 7S Framework     | Analyzes seven internal organizational elements (Strategy, Structure, Systems, Shared Values, Skills, Style, Staff) to ensure alignment and effectiveness in decision-making and change management. |
| 46     | Porter's Five Forces      | Examines competitive forces within an industry (Competitive Rivalry, Supplier Power, Buyer Power, Threat of Substitution, Threat of New Entry) to inform strategic decisions and market positioning. |
| 47     | Blue Ocean Strategy       | Encourages creating new market spaces (blue oceans) rather than competing in existing ones (red oceans), promoting differentiation and uncontested market creation.              |
| 48     | Appreciative Inquiry      | Focuses on identifying and leveraging an organization's strengths and successes to drive change and innovation, enhancing positive decision-making and organizational development. |
| 49     | Mind Mapping              | Utilizes visual diagrams to organize thoughts, ideas, and information, facilitating brainstorming, creativity, and problem-solving by illustrating relationships and hierarchies.   |
| 50     | Lean Six Sigma            | Combines Lean methodologies and Six Sigma principles to improve efficiency, reduce waste, and enhance quality in processes and decisions through data-driven approaches and continuous improvement. |


The prompt paired with the o1 model above returned a list of 50 Decision Frameworks (with descriptions) it is aware of. Presumably, because it knows of these decision frameworks, it should be able to operate on them to make decisions. This highlights just some of decision-making capability of the o1 series of model. What if there are more decision frameworks that are available? This is potentially just a list of the 50 (nice round number) of frameworks the model has identified. There could be a lot more.  

The code below will iterate several times (5x) and continue to prompt the GenAI model with the same instruction until a more comprehensive list is built.

In [10]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var markDownTaleOfDecisionFrameworks = string.Empty;

// This can quite a long time. View the output results below to see the final markdown table
for (int i = 0; i != 5; i++)
{
    var systemPromptForDecisions = $"""
    You are a Decision Intelligence Assistant. 
    Assist the user in exploring options, reasoning through decisions, problem-solving.
    Apply systems thinking to various scenarios. 
    Provide structured, logical, and comprehensive advice.
    """;
    var decisionAnalysisPrompt = $"""
    Identify and a very comprehensive list of decision-making frameworks that can enhance the quality of decisions. 
    Briefly describe how each framework supports better analysis and reasoning in different decision-making scenarios.

    Surface the gathered frameworks in a Markdown table format. Do not surround the markdown with triple backticks.
    The three table columns should be: Number (in ascending order), Decision-Framework Name and Decsription.
    Only return the Markdown table no other text.

    This prompt is going to be called several times to gather a full list of decision-making frameworks. 
    The previous Markdown table is located in the area surrounded by <DecisionMakingFrameworks> tags.
    Make sure to copy the table found in the previous responses and append new unique frameworks to the table. 
    DO NOT discard the previous Markdown table found in the <DecisionMakingFrameworks> tags.
    Add new unique frameworks by appending to the previous Markdown table. Do not recreate the table header.
    The final Markdown table output should include both:
    The previous markdown table (found in the <DecisionMakingFrameworks> Tags) combined with new unique frameworks output in a single table.

    <DecisionMakingFrameworks>
    {markDownTaleOfDecisionFrameworks}
    </DecisionMakingFrameworks>.
    """;

    // How the prompt looks like to the LLM
    var simpleDecisionPromptTemplate = $"""
    System Prompt: {systemPromptForDecisions}

    Request from the user: {decisionAnalysisPrompt}
    """;

    // Uncomment this line to view the debugging 
    // Console.WriteLine(decisionAnalysisPrompt);

    var simplePromptResponse = await semanticKernel.InvokePromptAsync(decisionAnalysisPrompt);
    // This will append the new markdown table to the previous markdown tables
    markDownTaleOfDecisionFrameworks = simplePromptResponse.GetValue<string>();
    Console.WriteLine($"Finished Gethering Intelligence Iteration: {i + 1}");
}

// Write the final markdown table to the console
Console.WriteLine(markDownTaleOfDecisionFrameworks);

Finished Gethering Intelligence Iteration: 1
Finished Gethering Intelligence Iteration: 2
Finished Gethering Intelligence Iteration: 3
Finished Gethering Intelligence Iteration: 4
Finished Gethering Intelligence Iteration: 5
| Number | Decision-Framework Name                         | Description |
|--------|-------------------------------------------------|-------------|
| 1      | SWOT Analysis                                   | Evaluates internal Strengths and Weaknesses, and external Opportunities and Threats to inform strategic planning and decision-making. |
| 2      | PEST Analysis                                   | Analyzes Political, Economic, Social, and Technological factors to understand the macro-environmental context affecting decisions. |
| 3      | Decision Trees                                  | Uses a tree-like model of decisions and their possible consequences to systematically evaluate options and outcomes. |
| 4      | Cost-Benefit Analysis                      

| Number | Decision-Framework Name                         | Description |
|--------|-------------------------------------------------|-------------|
| 1      | SWOT Analysis                                   | Evaluates internal Strengths and Weaknesses, and external Opportunities and Threats to inform strategic planning and decision-making. |
| 2      | PEST Analysis                                   | Analyzes Political, Economic, Social, and Technological factors to understand the macro-environmental context affecting decisions. |
| 3      | Decision Trees                                  | Uses a tree-like model of decisions and their possible consequences to systematically evaluate options and outcomes. |
| 4      | Cost-Benefit Analysis                           | Compares the costs and benefits of different options to determine the most financially advantageous choice. |
| 5      | Eisenhower Matrix                               | Prioritizes tasks by urgency and importance, helping to focus on what is most critical and manage time effectively. |
| 6      | Pareto Analysis                                 | Applies the 80/20 rule to identify the most significant factors contributing to a problem, aiding in prioritization. |
| 7      | Six Thinking Hats                               | Encourages looking at a problem from six distinct perspectives to enhance creativity and explore multiple angles in decision-making. |
| 8      | OODA Loop                                       | Stands for Observe, Orient, Decide, Act; a cyclical process for rapid decision-making in dynamic and fast-changing environments. |
| 9      | PDCA Cycle                                      | Plan-Do-Check-Act cycle used for continuous improvement and iterative problem-solving in processes and products. |
| 10     | Kepner-Tregoe Method                            | A systematic approach to problem-solving and decision-making by analyzing situations, clarifying objectives, and evaluating alternatives. |
| 11     | Force Field Analysis                            | Identifies and analyzes the forces for and against change, helping to weigh the pros and cons and make informed decisions. |
| 12     | Root Cause Analysis                             | A method used to identify the underlying causes of a problem, allowing for effective solutions by addressing root issues rather than symptoms. |
| 13     | RACI Matrix                                     | Clarifies roles and responsibilities in decision-making processes by assigning who is Responsible, Accountable, Consulted, and Informed. |
| 14     | Mind Mapping                                    | Visual tool that organizes information hierarchically, facilitating brainstorming and the exploration of connections between concepts in decision-making. |
| 15     | Delphi Method                                   | Gathers input from a panel of experts through multiple rounds of questionnaires to reach a consensus on complex decisions. |
| 16     | MoSCoW Method                                   | Prioritizes requirements or features into Must-haves, Should-haves, Could-haves, and Won't-haves to focus on what matters most in decision-making. |
| 17     | SCAMPER Technique                               | A creative thinking method that stands for Substitute, Combine, Adapt, Modify, Put to another use, Eliminate, and Reverse to generate innovative solutions. |
| 18     | The 5 Whys                                      | Iteratively asks "Why?" five times to drill down to the root cause of a problem, aiding in decision-making by uncovering fundamental issues. |
| 19     | Risk Analysis and Management                    | Identifies potential risks, assesses their impact, and develops strategies to manage them, supporting informed decision-making under uncertainty. |
| 20     | Fishbone Diagram (Ishikawa)                     | A visual tool that maps out potential causes of a problem to identify root causes and facilitate effective decision-making. |
| 21     | Ansoff Matrix                                   | Strategic planning tool that helps businesses decide their product and market growth strategy by focusing on existing and new products and markets. |
| 22     | Balanced Scorecard                              | Aligns business activities to the vision and strategy of the organization by monitoring performance across multiple perspectives for strategic decision-making. |
| 23     | Porter's Five Forces                            | Analyzes the competitive forces within an industry to assess its attractiveness and inform strategic decisions. |
| 24     | BCG Matrix                                      | Helps businesses analyze their product lines or business units by categorizing them into Stars, Cash Cows, Question Marks, and Dogs based on market growth and share. |
| 25     | SMART Goals                                     | Sets objectives that are Specific, Measurable, Achievable, Relevant, and Time-bound to provide clear direction for decision-making and planning. |
| 26     | Pros and Cons List                              | A simple yet effective method of listing the advantages and disadvantages of each option to aid in decision-making. |
| 27     | Multi-Criteria Decision Analysis (MCDA)         | Evaluates options based on multiple criteria weighted according to importance, allowing for complex decision-making with quantitative support. |
| 28     | SOAR Analysis                                   | Focuses on Strengths, Opportunities, Aspirations, and Results to create strategic plans that are positive and future-focused in decision-making. |
| 29     | Scenario Planning                               | Involves developing different plausible future scenarios to understand potential risks and opportunities to inform strategic decisions. |
| 30     | Value Chain Analysis                            | Analyzes the activities that create value to understand where improvements can be made and inform decisions to enhance competitive advantage. |
| 31     | Vroom-Yetton Decision Model                     | A leadership-focused model that provides a systematic approach to decision-making based on the level of subordinate involvement needed. |
| 32     | TRIZ (Theory of Inventive Problem Solving)      | A problem-solving tool derived from patterns of invention in the global patent literature, used to foster innovative solutions. |
| 33     | Monte Carlo Simulation                          | Uses randomness and statistical sampling to model complex systems and assess the impact of risk in quantitative analysis and decision-making. |
| 34     | Bayesian Decision Theory                        | Incorporates Bayesian statistics to update probability estimates as more evidence becomes available, aiding in decision-making under uncertainty. |
| 35     | Game Theory                                     | Analyzes strategic interactions where the outcome for each participant depends on the actions of all, helping in making decisions in competitive environments. |
| 36     | Theory of Constraints                           | Focuses on identifying and managing the most limiting factor (constraint) that stands in the way of achieving a goal, to improve performance. |
| 37     | Cynefin Framework                               | Helps leaders understand the context of a situation (simple, complicated, complex, chaotic) to adapt their decision-making approach accordingly. |
| 38     | Analytic Hierarchy Process (AHP)                | A structured technique for organizing and analyzing complex decisions using pairwise comparisons and priority scales. |
| 39     | DMAIC                                           | Define, Measure, Analyze, Improve, Control; a data-driven improvement cycle used for optimizing and stabilizing business processes and designs. |
| 40     | STOPE Model                                     | Assesses projects or strategies by evaluating Strategy, Technology, Organization, People, and Environment to ensure alignment and effectiveness. |
| 41     | Kotter's 8-Step Change Model                    | Provides a process for leading change by outlining eight steps to help leaders successfully implement transformations. |
| 42     | Ladder of Inference                             | Explains how individuals can move from observable data to actions, highlighting potential biases and promoting better reasoning. |
| 43     | LEAN Methodology                                | Focuses on maximizing customer value while minimizing waste, guiding decision-making in process improvements and operational efficiency. |
| 44     | Design Thinking                                 | An iterative approach used for practical and creative problem-solving with a user-centric focus, encouraging innovative solutions. |
| 45     | Business Model Canvas                           | A strategic management template for developing new or documenting existing business models, providing a holistic view for informed decision-making. |
| 46     | Critical Path Method (CPM)                      | A project management technique to identify critical and non-critical tasks, optimizing scheduling and resource allocation. |
| 47     | Theory U                                        | A framework for leading profound change by guiding individuals through a journey of sensing, presencing, and realizing for deep insights. |
| 48     | Blue Ocean Strategy                             | Encourages businesses to create uncontested market space ("blue oceans") rather than competing in saturated markets, guiding strategic decisions. |
| 49     | Quantitative Decision-Making Techniques         | Utilizes mathematical models and statistical techniques (e.g., linear programming) to optimize decisions based on quantitative data. |
| 50     | Appreciative Inquiry                            | Focuses on identifying and leveraging an organization's strengths, fostering positive decision-making and change management. |
| 51     | Rational Decision-Making Model                  | A step-by-step approach that involves identifying the problem, gathering information, evaluating alternatives, and choosing the optimal solution based on logical reasoning and analysis. |
| 52     | Ethical Decision-Making Model                   | Guides individuals to make decisions considering ethical principles and morals, assessing the rightness or wrongness of actions in complex situations. |
| 53     | Intuitive Decision-Making                       | Relies on instinct and gut feelings to make quick decisions without extensive analysis, often based on experience and pattern recognition. |
| 54     | Recognition-Primed Decision Model               | Combines intuition and analysis where decision-makers use their experience to recognize patterns and rapidly arrive at effective decisions in complex environments. |
| 55     | Cost-Effectiveness Analysis (CEA)               | Compares the relative costs and outcomes to determine which course of action provides the best results for the least cost. |
| 56     | Nudge Theory                                    | Utilizes positive reinforcement and indirect suggestions to influence behavior and decision-making, encouraging better choices without restricting options. |
| 57     | Heuristic Decision-Making                       | Employs mental shortcuts or rules of thumb to simplify decision-making processes, enabling quicker judgments with less cognitive effort. |
| 58     | Evidence-Based Decision Making                  | Involves using the best available data and empirical evidence to inform decision-making processes rather than relying solely on intuition. |
| 59     | System 1 and System 2 Thinking                  | Differentiates between fast, automatic thinking (System 1) and slow, deliberate thinking (System 2) in decision-making. |
| 60     | Expected Utility Theory                         | A mathematical approach to decision-making under uncertainty, aiming to maximize expected satisfaction based on probable outcomes. |
| 61     | Soft Systems Methodology (SSM)                  | A qualitative approach for tackling complex, human-centered problems by exploring different perceptions and defining solutions through learning. |
| 62     | Pre-mortem Analysis                             | A planning strategy where a team imagines a project has failed and works backward to identify potential risks and preventive measures. |
| 63     | Value-Focused Thinking                          | Emphasizes identifying and prioritizing underlying values before generating alternatives to ensure decisions align with key objectives. |
| 64     | Garbage Can Model                               | Describes decision-making in organizations as a chaotic process where problems, solutions, and participants are disconnected, resulting in random outcomes. |
| 65     | Minimax and Maximin Decision Rules              | Strategies for making decisions under uncertainty; Minimax minimizes the maximum loss, while Maximin maximizes the minimum gain. |
| 66     | Real Options Analysis                           | Applies financial options theory to investment decisions, valuing flexibility and strategic opportunities under uncertainty. |
| 67     | Consensus Decision-Making                       | A group decision-making process that seeks agreement from all participants, resolving objections to reach a collective decision. |
| 68     | Kaizen                                          | Focuses on continuous, incremental improvements involving all employees to enhance efficiency and quality in processes and decision-making. |
| 69     | Critical Thinking Framework                     | Encourages objective analysis and evaluation of issues to form a judgment, promoting logical reasoning in decision-making. |
| 70     | Cognitive Bias Awareness                        | Recognizes and mitigates cognitive biases that can distort perception and judgment, leading to more rational decisions. |
| 71     | Issue Tree Analysis                             | Breaks down complex problems into hierarchical components, facilitating systematic analysis and solution development. |
| 72     | McKinsey 7-S Framework                          | Analyzes organizational effectiveness by examining seven interrelated elements to ensure alignment in strategic decision-making. |
| 73     | KJ Method (Affinity Diagram)                    | Organizes large amounts of data into groupings based on relationships, enhancing understanding and clarity in decision-making. |
| 74     | Nominal Group Technique (NGT)                   | A structured group brainstorming method that encourages contributions from all members, followed by ranking to prioritize ideas. |
| 75     | Activity-Based Costing (ABC)                    | Assigns costs to activities based on resource usage, providing detailed insights for financial decision-making and process improvements. |
| 76     | Pugh Matrix                                     | Compares different design concepts against a baseline to identify the best option based on multiple criteria. |
| 77     | WRAP Model                                      | A four-step process: Widen your options, Reality-test your assumptions, Attain distance, Prepare to be wrong; designed to improve decision-making. |
| 78     | DACI Framework                                  | A decision-making model that clarifies roles by designating Drivers, Approvers, Contributors, and Informed parties. |
| 79     | Paired Comparison Analysis                      | Helps prioritize a list of options by comparing each option against all others in pairs to establish relative importance. |
| 80     | 7-Step Decision-Making Model                    | A process that involves identifying the decision, gathering information, identifying alternatives, weighing evidence, choosing, taking action, and reviewing the decision. |
| 81     | OSKAR Coaching Model                            | A solution-focused framework used in coaching that stands for Outcome, Scaling, Know-how & resources, Affirm & action, Review. |
| 82     | Decision Balance Sheet                          | Lists the benefits and costs associated with each option to help make informed decisions. |
| 83     | The 9 Windows Technique                         | Encourages thinking about a problem across time (past, present, future) and scale (subsystem, system, supersystem) to find innovative solutions. |
| 84     | PESTLE Analysis                                 | Extends PEST Analysis by including Environmental and Legal factors in addition to Political, Economic, Social, and Technological, aiding comprehensive decision-making. |
| 85     | RUMBA Criteria                                  | Ensures that goals and objectives are Realistic, Understandable, Measurable, Believable, and Attainable, aiding decision-making. |
| 86     | CLEAR Coaching Model                            | A framework for coaching conversations: Contract, Listen, Explore, Action, Review. Used to facilitate decision-making in personal development. |
| 87     | Gap Analysis                                    | Compares actual performance with potential or desired performance to identify gaps and inform strategic decisions. |
| 88     | Stakeholder Analysis                            | Identifies all parties affected by a decision, assesses their interests and influence, and helps in formulating strategies to manage stakeholders effectively. |
| 89     | Causal Loop Diagrams                            | A systems thinking tool to model the feedback loops within a system, helping to understand complex systems and make informed decisions. |
| 90     | Six Value Medals                                | Complements the Six Thinking Hats by focusing on values: Gold (human values), Silver (organizational values), Steel (quality), Glass (innovation), Wood (ecology), and Brass (perception). |
| 91     | SCQA Framework                                  | Situation, Complication, Question, Answer; a storytelling method used to structure communication and clarify decision points. |
| 92     | McKinsey's Three Horizons Model                 | A strategy framework for balancing short-term and long-term growth by categorizing initiatives into three horizons. |
| 93     | Grid Analysis                                   | Also known as a weighted scoring model, it helps in making decisions by scoring and weighting different options against criteria. |
| 94     | Four Frame Model                                | Analyzes organizations through Structural, Human Resource, Political, and Symbolic perspectives to inform management decisions. |
| 95     | Decision Quality Chain                          | Focuses on six elements—appropriate frame, creative alternatives, meaningful information, clear values and trade-offs, logically correct reasoning, commitment to action—to ensure sound decisions. |
| 96     | Hoshin Kanri Planning                           | A strategic planning method that aligns goals with middle management and operations, ensuring consistent decision-making across all levels. |
| 97     | Value Proposition Canvas                        | Helps ensure a product or service is positioned around what the customer values and needs, guiding product development decisions. |
| 98     | Johari Window                                   | A model to understand self-awareness in interpersonal relationships, aiding in personal development decision-making. |
| 99     | RED Model                                       | Recognize Assumptions, Evaluate Arguments, Draw Conclusions; a framework to enhance critical thinking and decision-making. |
| 100    | Plus-Minus-Interesting (PMI) Analysis           | Involves listing the positive, negative, and interesting points of an idea to assess its potential impacts and make balanced decisions. |